In [1]:
import hopsworks
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

/opt/anaconda3/envs/SMDL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/194711
Connected. Call `.close()` to terminate connection gracefully.


In [21]:
# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view
buoy_swells_huntington_fg = fs.get_feature_group(name="buoy_swells_huntington", version=1)
query = buoy_swells_huntington_fg.select_all()
df = query.read()
df['hits_at'] = df['hits_at'].dt.strftime('%Y-%m-%d %H')
buoy_swells_sorted = df.sort_values(by='hits_at', ascending=False)
buoy_swells_sorted

Finished: Reading data from Hopsworks, using ArrowFlight (1.68s) 


,pred_dtime,height,period,direction,hits_at,beach_id
576,2024-01-03 12:00:00+00:00,0.54,4.0,14,2024-01-14 04,1
581,2024-01-03 12:00:00+00:00,0.16,15.3,12,2024-01-14 03,1
580,2024-01-03 12:00:00+00:00,0.16,15.2,12,2024-01-14 03,1
575,2024-01-03 12:00:00+00:00,1.09,13.3,77,2024-01-14 02,1
579,2024-01-03 12:00:00+00:00,1.11,13.4,78,2024-01-14 01,1
...,...,...,...,...,...,...
251,2023-12-29 00:00:00+00:00,1.43,17.1,89,2023-12-29 03,1
86,2023-12-29 00:00:00+00:00,1.46,17.4,89,2023-12-29 02,1
284,2023-12-29 00:00:00+00:00,1.46,17.6,89,2023-12-29 01,1
365,2023-12-29 00:00:00+00:00,1.45,17.7,89,2023-12-29 01,1


In [22]:
beach_swells_huntington_fg = fs.get_feature_group(name="beach_swells_huntington", version=1)
query2 = beach_swells_huntington_fg.select_all()
df2 = query2.read()
df2['datetime'] = df2['datetime'].dt.strftime('%Y-%m-%d %H')
beach_swells_sorted = df2.sort_values(by='datetime', ascending=False)
beach_swells_sorted

Finished: Reading data from Hopsworks, using ArrowFlight (0.61s) 


,quality,datetime,year,month,day,hour,minute
29,Poor,2024-01-03 12,2024,1,3,12,30
28,Poor,2024-01-03 08,2024,1,3,8,0
1,Fair,2024-01-02 12,2024,1,2,12,30
25,Poor To Fair,2024-01-02 08,2024,1,2,8,0
10,Fair,2024-01-01 12,2024,1,1,12,30
13,Fair,2024-01-01 08,2024,1,1,8,0
21,Fair,2023-12-31 12,2023,12,31,12,30
24,Fair,2023-12-31 08,2023,12,31,8,0
12,Poor,2023-12-30 12,2023,12,30,12,30
11,Poor To Fair,2023-12-30 08,2023,12,30,8,0


In [31]:
merged_df = pd.merge(buoy_swells_sorted, beach_swells_sorted, left_on='hits_at', right_on='datetime', how='inner')
merged_df = merged_df.drop(["pred_dtime",'datetime'],axis=1)
merged_df

,height,period,direction,hits_at,beach_id,quality,year,month,day,hour,minute
0,1.64,5.6,94,2024-01-03 12,1,Poor,2024,1,3,12,30
1,0.35,4.0,82,2024-01-03 08,1,Poor,2024,1,3,8,0
2,0.56,11.3,80,2024-01-02 12,1,Fair,2024,1,2,12,30
3,0.19,12.7,13,2024-01-02 08,1,Poor To Fair,2024,1,2,8,0
4,0.20,13.5,14,2024-01-01 12,1,Fair,2024,1,1,12,30
5,0.21,14.1,13,2024-01-01 08,1,Fair,2024,1,1,8,0
6,0.20,14.1,11,2023-12-31 12,1,Fair,2023,12,31,12,30
7,0.20,14.6,10,2023-12-31 08,1,Fair,2023,12,31,8,0
8,0.19,15.5,11,2023-12-30 12,1,Poor,2023,12,30,12,30
9,0.18,16.0,13,2023-12-30 08,1,Poor To Fair,2023,12,30,8,0


In [36]:
# Save the merged DataFrame as a new Feature Group
merged_fg = fs.get_or_create_feature_group(name="merged_swells_huntington",
                version=1,
                primary_key=["beach_id"],
                description="Merged swell from buoy and huntington website",
                online_enabled=True,
                statistics_config={"enabled": True, "histograms": True, "correlations": True},
                )
merged_fg.insert(merged_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/194711/fs/194630/fg/384364


Uploading Dataframe: 100.00% |██████████| Rows 12/12 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: merged_swells_huntington_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/194711/jobs/named/merged_swells_huntington_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fe3d9a03dc0>, None)

In [38]:
# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view
merged_fg = fs.get_feature_group(name="merged_swells_huntington", version=1)
query = merged_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="merged_swells_huntington",
                                  version=1,
                                  description="Feature view combining the buoy swells and the beach swells",
                                  labels=["quality"],
                                  query=query)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/194711/fs/194630/fv/merged_swells_huntington/version/1
